# %%
# =============================================================================
# INCOME PREDICTION MODEL - CLEAN ML PIPELINE
# =============================================================================
# Goal: Predict customer income (ingresos_reportados) using enhanced features
# Current Performance: R² ≈ 0.31 (Target: Improve to 0.35+)
# Models: XGBoost, LightGBM, Random Forest with GroupKFold CV
# =============================================================================


In [1]:
# %%
# SECTION 1: IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GroupKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("=" * 80)
print("INCOME PREDICTION MODEL - CLEAN PIPELINE STARTED")
print("=" * 80)

INCOME PREDICTION MODEL - CLEAN PIPELINE STARTED


In [2]:
# %%
# SECTION 2: DATA LOADING AND INITIAL SETUP
# =============================================================================
print("\n🔄 LOADING PROCESSED DATA")
print("-" * 50)

# Define data path
data_path = r'C:\Users\david\OneDrive\Documents\augment-projects\caja-de-ahorros\data\processed'

# Load the main dataset
df_original = pd.read_csv(data_path + '/df_clientes_clean.csv')

print(f"✅ Original dataset loaded: {df_original.shape}")
print(f"   Columns: {df_original.shape[1]}")
print(f"   Records: {df_original.shape[0]:,}")


🔄 LOADING PROCESSED DATA
--------------------------------------------------
✅ Original dataset loaded: (15000, 32)
   Columns: 32
   Records: 15,000


In [ ]:
# %%
# SECTION 3: FEATURE SELECTION AND DATASET PREPARATION
# =============================================================================
print("\n🎯 PREPARING FEATURE SET")
print("-" * 50)

# Define columns to keep for modeling
columns_to_keep = [
    # ID columns
    'cliente', 'identificador_unico',
    
    # Target variable
    'ingresos_reportados',
    
    # Core features (already processed)
    'edad', 'letras_mensuales', 'monto_letra', 'saldo',
    'fechaingresoempleo', 'fecha_inicio', 'fecha_vencimiento',
    'ocupacion_consolidated', 'ciudad_consolidated',
    'nombreempleadorcliente_consolidated', 'cargoempleocliente_consolidated',
    'sexo_consolidated', 'estado_civil_consolidated', 'pais_consolidated',
    'missing_fechaingresoempleo', 'missing_nombreempleadorcliente', 'missing_cargoempleocliente',
    'is_retired', 'age_group'
]

# Filter to existing columns
existing_columns = [col for col in columns_to_keep if col in df_original.columns]
df_working = df_original[existing_columns].copy()

print(f"✅ Working dataset prepared: {df_working.shape}")
print(f"   Features to process: {len(existing_columns) - 3}")  # Exclude IDs and target


In [ ]:
# %%
# SECTION 4: DATA PREPROCESSING PIPELINE
# =============================================================================
print("\n🔧 DATA PREPROCESSING PIPELINE")
print("-" * 50)

def preprocess_data(df):
    """
    Complete data preprocessing pipeline
    """
    df_processed = df.copy()
    
    # 4.1: Handle Missing Values
    print("   📋 Handling missing values...")
    
    # Numerical missing values
    if 'monto_letra' in df_processed.columns:
        df_processed['monto_letra_missing'] = df_processed['monto_letra'].isnull().astype(int)
        median_monto = df_processed['monto_letra'].median()
        df_processed['monto_letra'] = df_processed['monto_letra'].fillna(median_monto)
        print(f"      ✅ monto_letra: filled {df_processed['monto_letra_missing'].sum()} missing values")
    
    # 4.2: Convert Date Features
    print("   📅 Converting date features...")
    reference_date = pd.Timestamp('2020-01-01')
    date_columns = ['fechaingresoempleo', 'fecha_inicio', 'fecha_vencimiento']
    
    for date_col in date_columns:
        if date_col in df_processed.columns:
            # Convert to datetime
            df_processed[date_col] = pd.to_datetime(df_processed[date_col], errors='coerce')
            
            # Convert to days since reference
            df_processed[f'{date_col}_days'] = (df_processed[date_col] - reference_date).dt.days
            
            # Handle missing values
            missing_count = df_processed[f'{date_col}_days'].isnull().sum()
            if missing_count > 0:
                df_processed[f'{date_col}_missing'] = df_processed[f'{date_col}_days'].isnull().astype(int)
                median_days = df_processed[f'{date_col}_days'].median()
                df_processed[f'{date_col}_days'] = df_processed[f'{date_col}_days'].fillna(median_days)
                print(f"      ✅ {date_col}: converted to days, filled {missing_count} missing")
            
            # Drop original date column
            df_processed.drop(columns=[date_col], inplace=True)
    
    # 4.3: Encode Categorical Features
    print("   🏷️  Encoding categorical features...")
    
    # Low cardinality - One-hot encoding
    low_cardinality = ['sexo_consolidated', 'estado_civil_consolidated', 'pais_consolidated', 'age_group']
    existing_low_card = [col for col in low_cardinality if col in df_processed.columns]
    
    for col in existing_low_card:
        df_processed[col] = df_processed[col].fillna('Unknown')
        dummies = pd.get_dummies(df_processed[col], prefix=col, drop_first=True)
        df_processed = pd.concat([df_processed, dummies], axis=1)
        df_processed.drop(columns=[col], inplace=True)
        print(f"      ✅ {col}: created {len(dummies.columns)} dummy variables")
    
    # High cardinality - Frequency encoding
    high_cardinality = ['ocupacion_consolidated', 'ciudad_consolidated',
                       'nombreempleadorcliente_consolidated', 'cargoempleocliente_consolidated']
    existing_high_card = [col for col in high_cardinality if col in df_processed.columns]
    
    for col in existing_high_card:
        df_processed[col] = df_processed[col].fillna('Unknown')
        freq_map = df_processed[col].value_counts().to_dict()
        new_col_name = f'{col}_freq'
        df_processed[new_col_name] = df_processed[col].map(freq_map)
        df_processed.drop(columns=[col], inplace=True)
        print(f"      ✅ {col}: frequency encoded (range: {df_processed[new_col_name].min()}-{df_processed[new_col_name].max()})")
    
    return df_processed

# Apply preprocessing
df_processed = preprocess_data(df_working)
print(f"\n✅ Preprocessing complete: {df_processed.shape}")

In [ ]:
# %%
# SECTION 5: FEATURE ENGINEERING FUNCTION
# =============================================================================
print("\n⚙️ FEATURE ENGINEERING SETUP")
print("-" * 50)

def create_interaction_features(df):
    """
    Create interaction features for enhanced model performance
    """
    df_enhanced = df.copy()
    
    print(f"   🔧 Creating interaction features for dataset: {df.shape}")
    
    # Get age group columns
    age_columns = [col for col in df.columns if col.startswith('age_group_')]
    
    # 1. Age-Occupation Interactions
    if 'ocupacion_consolidated_freq' in df.columns and age_columns:
        for age_col in age_columns:
            interaction_name = f"{age_col}_x_occupation"
            df_enhanced[interaction_name] = (df_enhanced[age_col] * df_enhanced['ocupacion_consolidated_freq'])
        print(f"      ✅ Created {len(age_columns)} age-occupation interactions")
    
    # 2. Location-Occupation Interaction
    if ('ciudad_consolidated_freq' in df.columns and 'ocupacion_consolidated_freq' in df.columns):
        df_enhanced['location_x_occupation'] = (df_enhanced['ciudad_consolidated_freq'] * 
                                              df_enhanced['ocupacion_consolidated_freq'])
        print(f"      ✅ Created location-occupation interaction")
    
    # 3. Retirement-Age Interactions
    if 'is_retired' in df.columns and age_columns:
        for age_col in age_columns:
            interaction_name = f"retired_x_{age_col}"
            df_enhanced[interaction_name] = (df_enhanced['is_retired'] * df_enhanced[age_col])
        print(f"      ✅ Created {len(age_columns)} retirement-age interactions")
    
    # 4. Employer-Position Interaction
    if ('nombreempleadorcliente_consolidated_freq' in df.columns and 
        'cargoempleocliente_consolidated_freq' in df.columns):
        df_enhanced['employer_x_position'] = (df_enhanced['nombreempleadorcliente_consolidated_freq'] * 
                                            df_enhanced['cargoempleocliente_consolidated_freq'])
        print(f"      ✅ Created employer-position interaction")
    
    # 5. Gender-Occupation Interaction
    if ('sexo_consolidated_Masculino' in df.columns and 'ocupacion_consolidated_freq' in df.columns):
        df_enhanced['gender_x_occupation'] = (df_enhanced['sexo_consolidated_Masculino'] * 
                                            df_enhanced['ocupacion_consolidated_freq'])
        print(f"      ✅ Created gender-occupation interaction")
    
    # 6. High-Value Combinations
    if 'ocupacion_consolidated_freq' in df.columns:
        occupation_75th = df_enhanced['ocupacion_consolidated_freq'].quantile(0.75)
        df_enhanced['high_freq_occupation'] = (df_enhanced['ocupacion_consolidated_freq'] >= occupation_75th).astype(int)
    
    if 'ciudad_consolidated_freq' in df.columns:
        city_75th = df_enhanced['ciudad_consolidated_freq'].quantile(0.75)
        df_enhanced['high_freq_city'] = (df_enhanced['ciudad_consolidated_freq'] >= city_75th).astype(int)
    
    if ('high_freq_occupation' in df_enhanced.columns and 'high_freq_city' in df_enhanced.columns):
        df_enhanced['high_value_location_job'] = (df_enhanced['high_freq_occupation'] * 
                                                df_enhanced['high_freq_city'])
        print(f"      ✅ Created high-value feature combinations")
    
    # 7. FINANCIAL BEHAVIOR FEATURES
    # ============================================================================
    print("   💰 Creating financial behavior features...")
    
    # Debt-to-income ratio (if monto_letra exists)
    if 'monto_letra' in df.columns and 'edad' in df.columns:
        # Payment burden relative to age
        df_enhanced['payment_per_age'] = df_enhanced['monto_letra'] / (df_enhanced['edad'] + 1)
        
        # High payment burden indicator
        payment_75th = df_enhanced['monto_letra'].quantile(0.75)
        df_enhanced['high_payment_burden'] = (df_enhanced['monto_letra'] >= payment_75th).astype(int)
        print(f"      ✅ Created payment burden features")
    
    # Account balance features
    if 'saldo' in df.columns:
        # Balance categories
        saldo_25th = df_enhanced['saldo'].quantile(0.25)
        saldo_75th = df_enhanced['saldo'].quantile(0.75)
        
        df_enhanced['low_balance'] = (df_enhanced['saldo'] <= saldo_25th).astype(int)
        df_enhanced['high_balance'] = (df_enhanced['saldo'] >= saldo_75th).astype(int)
        df_enhanced['medium_balance'] = ((df_enhanced['saldo'] > saldo_25th) & 
                                       (df_enhanced['saldo'] < saldo_75th)).astype(int)
        print(f"      ✅ Created balance category features")
    
    # Payment frequency features
    if 'letras_mensuales' in df.columns:
        # Payment frequency categories
        df_enhanced['low_frequency_payments'] = (df_enhanced['letras_mensuales'] <= 1).astype(int)
        df_enhanced['medium_frequency_payments'] = ((df_enhanced['letras_mensuales'] > 1) & 
                                                  (df_enhanced['letras_mensuales'] <= 3)).astype(int)
        df_enhanced['high_frequency_payments'] = (df_enhanced['letras_mensuales'] > 3).astype(int)
        print(f"      ✅ Created payment frequency features")
    
    # 8. EMPLOYMENT STABILITY FEATURES
    # ============================================================================
    print("   🏢 Creating employment stability features...")
    
    # Employment tenure (if date features exist)
    date_cols = [col for col in df.columns if col.endswith('_days')]
    if any('fechaingresoempleo' in col for col in date_cols):
        employment_col = [col for col in date_cols if 'fechaingresoempleo' in col][0]
        
        # Employment tenure categories (assuming more recent dates = higher values)
        tenure_25th = df_enhanced[employment_col].quantile(0.25)
        tenure_75th = df_enhanced[employment_col].quantile(0.75)
        
        df_enhanced['short_tenure'] = (df_enhanced[employment_col] <= tenure_25th).astype(int)
        df_enhanced['long_tenure'] = (df_enhanced[employment_col] >= tenure_75th).astype(int)
        df_enhanced['medium_tenure'] = ((df_enhanced[employment_col] > tenure_25th) & 
                                      (df_enhanced[employment_col] < tenure_75th)).astype(int)
        print(f"      ✅ Created employment tenure features")
    
    # Employer size categories
    if 'nombreempleadorcliente_consolidated_freq' in df.columns:
        employer_median = df_enhanced['nombreempleadorcliente_consolidated_freq'].median()
        df_enhanced['large_employer'] = (df_enhanced['nombreempleadorcliente_consolidated_freq'] >= employer_median).astype(int)
        df_enhanced['small_employer'] = (df_enhanced['nombreempleadorcliente_consolidated_freq'] < employer_median).astype(int)
        print(f"      ✅ Created employer size features")
    
    # 9. DEMOGRAPHIC COMBINATIONS
    # ============================================================================
    print("   👥 Creating demographic combination features...")
    
    # Gender-Age interactions
    if 'sexo_consolidated_Masculino' in df.columns and age_columns:
        for age_col in age_columns:
            df_enhanced[f"male_x_{age_col}"] = (df_enhanced['sexo_consolidated_Masculino'] * 
                                              df_enhanced[age_col])
        print(f"      ✅ Created {len(age_columns)} gender-age interactions")
    
    # Marital status interactions
    marital_cols = [col for col in df.columns if col.startswith('estado_civil_')]
    if marital_cols and age_columns:
        for marital_col in marital_cols:
            for age_col in age_columns:
                interaction_name = f"{marital_col}_x_{age_col}"
                df_enhanced[interaction_name] = (df_enhanced[marital_col] * df_enhanced[age_col])
        print(f"      ✅ Created {len(marital_cols) * len(age_columns)} marital-age interactions")
    
    # 10. LOCATION-BASED FEATURES
    # ============================================================================
    print("   🌍 Creating location-based features...")
    
    # City size categories
    if 'ciudad_consolidated_freq' in df.columns:
        city_median = df_enhanced['ciudad_consolidated_freq'].median()
        city_75th = df_enhanced['ciudad_consolidated_freq'].quantile(0.75)
        
        df_enhanced['major_city'] = (df_enhanced['ciudad_consolidated_freq'] >= city_75th).astype(int)
        df_enhanced['medium_city'] = ((df_enhanced['ciudad_consolidated_freq'] >= city_median) & 
                                    (df_enhanced['ciudad_consolidated_freq'] < city_75th)).astype(int)
        df_enhanced['small_city'] = (df_enhanced['ciudad_consolidated_freq'] < city_median).astype(int)
        print(f"      ✅ Created city size category features")
    
    # Location-Gender interactions
    if ('ciudad_consolidated_freq' in df.columns and 'sexo_consolidated_Masculino' in df.columns):
        df_enhanced['male_x_city_freq'] = (df_enhanced['sexo_consolidated_Masculino'] * 
                                         df_enhanced['ciudad_consolidated_freq'])
        print(f"      ✅ Created location-gender interaction")
    
    # 11. RISK PROFILE FEATURES
    # ============================================================================
    print("   ⚠️ Creating risk profile features...")
    
    # Age-based risk categories
    if 'edad' in df.columns:
        df_enhanced['young_adult'] = ((df_enhanced['edad'] >= 18) & (df_enhanced['edad'] <= 30)).astype(int)
        df_enhanced['prime_age'] = ((df_enhanced['edad'] > 30) & (df_enhanced['edad'] <= 50)).astype(int)
        df_enhanced['senior'] = (df_enhanced['edad'] > 50).astype(int)
        print(f"      ✅ Created age-based risk categories")
    
    # Combined risk indicators
    risk_features = []
    if 'is_retired' in df.columns:
        risk_features.append('is_retired')
    if 'high_payment_burden' in df_enhanced.columns:
        risk_features.append('high_payment_burden')
    if 'low_balance' in df_enhanced.columns:
        risk_features.append('low_balance')
    if 'short_tenure' in df_enhanced.columns:
        risk_features.append('short_tenure')
    
    if len(risk_features) >= 2:
        # Create risk score (sum of risk indicators)
        df_enhanced['risk_score'] = df_enhanced[risk_features].sum(axis=1)
        df_enhanced['high_risk_profile'] = (df_enhanced['risk_score'] >= 2).astype(int)
        print(f"      ✅ Created combined risk profile features")
    
    # 12. TEMPORAL FEATURES
    # ============================================================================
    print("   📅 Creating temporal features...")
    
    # Contract duration (if start and end dates exist)
    start_date_cols = [col for col in df.columns if 'fecha_inicio' in col and col.endswith('_days')]
    end_date_cols = [col for col in df.columns if 'fecha_vencimiento' in col and col.endswith('_days')]
    
    if start_date_cols and end_date_cols:
        start_col = start_date_cols[0]
        end_col = end_date_cols[0]
        
        df_enhanced['contract_duration'] = df_enhanced[end_col] - df_enhanced[start_col]
        
        # Duration categories
        duration_median = df_enhanced['contract_duration'].median()
        df_enhanced['short_contract'] = (df_enhanced['contract_duration'] <= duration_median).astype(int)
        df_enhanced['long_contract'] = (df_enhanced['contract_duration'] > duration_median).astype(int)
        print(f"      ✅ Created contract duration features")
    
    # 13. INTERACTION COMPLEXITY FEATURES
    # ============================================================================
    print("   🔗 Creating complex interaction features...")
    
    # Three-way interactions (most predictive combinations)
    if ('ocupacion_consolidated_freq' in df.columns and 
        'ciudad_consolidated_freq' in df.columns and 
        'edad' in df.columns):
        
        # Normalize features for interaction
        occ_norm = df_enhanced['ocupacion_consolidated_freq'] / df_enhanced['ocupacion_consolidated_freq'].max()
        city_norm = df_enhanced['ciudad_consolidated_freq'] / df_enhanced['ciudad_consolidated_freq'].max()
        age_norm = df_enhanced['edad'] / df_enhanced['edad'].max()
        
        df_enhanced['occupation_city_age_interaction'] = occ_norm * city_norm * age_norm
        print(f"      ✅ Created three-way interaction feature")
    
    # Professional profile combinations
    if ('ocupacion_consolidated_freq' in df.columns and 
        'nombreempleadorcliente_consolidated_freq' in df.columns and
        'cargoempleocliente_consolidated_freq' in df.columns):
        
        # Professional stability score
        occ_norm = df_enhanced['ocupacion_consolidated_freq'] / df_enhanced['ocupacion_consolidated_freq'].max()
        emp_norm = df_enhanced['nombreempleadorcliente_consolidated_freq'] / df_enhanced['nombreempleadorcliente_consolidated_freq'].max()
        pos_norm = df_enhanced['cargoempleocliente_consolidated_freq'] / df_enhanced['cargoempleocliente_consolidated_freq'].max()
        
        df_enhanced['professional_stability_score'] = (occ_norm + emp_norm + pos_norm) / 3
        df_enhanced['high_professional_stability'] = (df_enhanced['professional_stability_score'] >= 0.7).astype(int)
        print(f"      ✅ Created professional stability features")
    
    new_features = [col for col in df_enhanced.columns if col not in df.columns]
    print(f"   ✨ Feature engineering complete: +{len(new_features)} new features")
    
    return df_enhanced

print("✅ Feature engineering function ready")

In [ ]:
# %%
# SECTION 6: TRAIN/VALIDATION/TEST SPLIT (NO ID OVERLAP)
# =============================================================================
print("\n📊 CREATING TRAIN/VALIDATION/TEST SPLITS")
print("-" * 50)

from sklearn.model_selection import train_test_split

# Define split ratios
TRAIN_RATIO, VALID_RATIO, TEST_RATIO = 0.70, 0.15, 0.15

# Split by customer IDs to prevent data leakage
unique_customers = df_processed['identificador_unico'].unique()
print(f"   Total customers: {len(unique_customers):,}")

# First split: separate test customers
train_valid_customers, test_customers = train_test_split(
    unique_customers, test_size=TEST_RATIO, random_state=42, shuffle=True)

# Second split: separate train and validation customers
train_customers, valid_customers = train_test_split(
    train_valid_customers, test_size=VALID_RATIO/(TRAIN_RATIO + VALID_RATIO),
    random_state=42, shuffle=True)

# Create datasets based on customer splits
train_df = df_processed[df_processed['identificador_unico'].isin(train_customers)].copy()
valid_df = df_processed[df_processed['identificador_unico'].isin(valid_customers)].copy()
test_df = df_processed[df_processed['identificador_unico'].isin(test_customers)].copy()

print(f"   ✅ Train: {len(train_df):,} records ({len(train_df)/len(df_processed):.1%})")
print(f"   ✅ Valid: {len(valid_df):,} records ({len(valid_df)/len(df_processed):.1%})")
print(f"   ✅ Test: {len(test_df):,} records ({len(test_df)/len(df_processed):.1%})")

# Verify no customer overlap
train_ids = set(train_df['identificador_unico'].unique())
valid_ids = set(valid_df['identificador_unico'].unique())
test_ids = set(test_df['identificador_unico'].unique())

overlaps = [
    len(train_ids.intersection(valid_ids)),
    len(train_ids.intersection(test_ids)),
    len(valid_ids.intersection(test_ids))
]

if sum(overlaps) == 0:
    print("   ✅ No customer ID overlap - data leakage prevented")
else:
    print("   ❌ Customer overlap detected!")

In [ ]:
# %%
# SECTION 7: OUTLIER CLEANING (TRAINING-BASED PARAMETERS)
# =============================================================================
print("\n🧹 OUTLIER CLEANING")
print("-" * 50)

def clean_target_outliers(df_list, target_col='ingresos_reportados'):
    """
    Clean outliers using training set parameters to prevent data leakage
    """
    # Get outlier parameters from training set only
    train_target = df_list[0][target_col]
    lower_cap = train_target.quantile(0.01)  # 1st percentile
    upper_cap = train_target.quantile(0.99)  # 99th percentile

    print(f"   📏 Outlier bounds from training: ${lower_cap:,.2f} to ${upper_cap:,.2f}")

    cleaned_dfs = []
    set_names = ['Training', 'Validation', 'Test']

    for i, (df, set_name) in enumerate(zip(df_list, set_names)):
        df_clean = df.copy()
        original_target = df_clean[target_col].copy()

        # Apply winsorization using training parameters
        df_clean[target_col] = original_target.clip(lower=lower_cap, upper=upper_cap)

        # Count changes
        n_capped = (original_target != df_clean[target_col]).sum()
        print(f"   ✅ {set_name}: {n_capped:,} values capped ({n_capped/len(df)*100:.1f}%)")

        # Add outlier flags
        df_clean['was_winsorized'] = (original_target != df_clean[target_col]).astype(int)

        cleaned_dfs.append(df_clean)

    return cleaned_dfs

# Apply outlier cleaning
train_df_clean, valid_df_clean, test_df_clean = clean_target_outliers([train_df, valid_df, test_df])

print("   ✅ Outlier cleaning complete - consistent across all sets")


In [ ]:
# %%
# SECTION 8: FEATURE ENGINEERING APPLICATION
# =============================================================================
print("\n⚙️ APPLYING FEATURE ENGINEERING")
print("-" * 50)

# Apply feature engineering to all datasets
train_df_enhanced = create_interaction_features(train_df_clean)
valid_df_enhanced = create_interaction_features(valid_df_clean)
test_df_enhanced = create_interaction_features(test_df_clean)

print(f"\n🎯 ENHANCED DATASET SHAPES:")
print(f"   Training: {train_df_enhanced.shape}")
print(f"   Validation: {valid_df_enhanced.shape}")
print(f"   Test: {test_df_enhanced.shape}")

In [ ]:
# %%
# SECTION 9: PREPARE FINAL DATASETS FOR MODELING
# =============================================================================
print("\n🎯 PREPARING FINAL DATASETS")
print("-" * 50)

# RECOMMENDED APPROACH
id_columns = ['cliente', 'identificador_unico']
target_column = 'ingresos_reportados'

# List features you want to drop
features_to_drop = [
    'outlier_iqr',
    'outlier_domain', 
    'was_winsorized',
    # Add any other features you want to exclude
]

# Create final feature list
columns_to_exclude = id_columns + [target_column] + features_to_drop
feature_columns = [col for col in train_df_enhanced.columns
                  if col not in columns_to_exclude]



#feature_columns = [col for col in train_df_enhanced.columns
 #                 if col not in id_columns + [target_column]]

print(f"   📊 Feature columns: {len(feature_columns)}")
print(f"   🎯 Target column: {target_column}")

# Create feature matrices and targets
X_train = train_df_enhanced[feature_columns].copy()
y_train = train_df_enhanced[target_column].copy()

X_valid = valid_df_enhanced[feature_columns].copy()
y_valid = valid_df_enhanced[target_column].copy()

X_test = test_df_enhanced[feature_columns].copy()
y_test = test_df_enhanced[target_column].copy()

print(f"\n📈 FINAL DATASET SHAPES:")
print(f"   X_train: {X_train.shape}")
print(f"   X_valid: {X_valid.shape}")
print(f"   X_test: {X_test.shape}")

# Verify data quality
print(f"\n✅ DATA QUALITY CHECKS:")
print(f"   Missing values in X_train: {X_train.isnull().sum().sum()}")
print(f"   Missing values in y_train: {y_train.isnull().sum()}")
print(f"   All features numeric: {all(X_train.dtypes.apply(lambda x: x in ['int64', 'float64']))}")


In [ ]:
# %%
# INSPECT FINAL FEATURE NAMES
# =============================================================================
print(f"\n📋 FINAL FEATURE LIST ({len(feature_columns)} features):")
print("-" * 60)

# Group features by type for better readability
basic_features = []
age_features = []
freq_features = []
interaction_features = []
other_features = []

for feature in feature_columns:
    if feature.startswith('age_group_'):
        age_features.append(feature)
    elif feature.endswith('_freq'):
        freq_features.append(feature)
    elif '_x_' in feature or 'retired_x_' in feature or 'employer_x_' in feature or 'gender_x_' in feature:
        interaction_features.append(feature)
    elif feature in ['edad', 'letras_mensuales', 'monto_letra', 'saldo', 'is_retired']:
        basic_features.append(feature)
    else:
        other_features.append(feature)

print(f"🔢 BASIC FEATURES ({len(basic_features)}):")
for feature in basic_features:
    print(f"   - {feature}")

print(f"\n👥 AGE GROUP FEATURES ({len(age_features)}):")
for feature in age_features:
    print(f"   - {feature}")

print(f"\n📊 FREQUENCY FEATURES ({len(freq_features)}):")
for feature in freq_features:
    print(f"   - {feature}")

print(f"\n⚡ INTERACTION FEATURES ({len(interaction_features)}):")
for feature in interaction_features:
    print(f"   - {feature}")

print(f"\n🔧 OTHER FEATURES ({len(other_features)}):")
for feature in other_features:
    print(f"   - {feature}")

# Save feature list to file for reference
feature_list_df = pd.DataFrame({
    'feature_name': feature_columns,
    'feature_type': ['basic' if f in basic_features else
                    'age_group' if f in age_features else
                    'frequency' if f in freq_features else
                    'interaction' if f in interaction_features else
                    'other' for f in feature_columns]
})

feature_list_df.to_csv(data_path + '/final_feature_list.csv', index=False)
print(f"\n💾 Feature list saved to: final_feature_list.csv")

In [ ]:
# %%
# SECTION 10: GROUPKFOLD SETUP FOR IMBALANCED FEATURES
# =============================================================================
print("\n🔄 GROUPKFOLD SETUP FOR IMBALANCED FEATURES")
print("-" * 50)

def create_age_groups(df):
    """
    Create groups based on imbalanced age features for GroupKFold
    """
    imbalanced_age_features = ['age_group_26-35', 'age_group_36-45', 'age_group_65+']
    groups = []

    for idx, row in df.iterrows():
        group_assigned = False
        for i, age_feature in enumerate(imbalanced_age_features):
            if age_feature in df.columns and row[age_feature] == 1:
                groups.append(i + 1)  # Groups 1, 2, 3
                group_assigned = True
                break

        if not group_assigned:
            groups.append(0)  # Default group

    return np.array(groups)

# Create groups for training set
groups_train = create_age_groups(X_train)
group_counts = pd.Series(groups_train).value_counts().sort_index()

print(f"   📊 Group distribution:")
for group, count in group_counts.items():
    print(f"      Group {group}: {count:,} samples ({count/len(groups_train)*100:.1f}%)")

# Setup GroupKFold
gkf = GroupKFold(n_splits=4)
print(f"   ✅ GroupKFold configured: 4 splits")

print("\n🚀 READY FOR MODEL TRAINING!")
print("=" * 80)

In [ ]:
# %%
# SECTION 11: MODEL TRAINING AND COMPARISON
# =============================================================================
print("\n🤖 MODEL TRAINING AND COMPARISON")
print("-" * 50)

# Apply robust scaling to features
print("   ⚖️ Applying RobustScaler...")
scaler = RobustScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_valid_scaled = pd.DataFrame(
    scaler.transform(X_valid),
    columns=X_valid.columns,
    index=X_valid.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)
print("   ✅ Feature scaling complete")

# IMMEDIATE ACTION: Replace your model configs with this
models = {
    'XGBoost': xgb.XGBRegressor(
        n_estimators=500, max_depth=8, learning_rate=0.05,
        subsample=0.85, colsample_bytree=0.85, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=500, max_depth=8, learning_rate=0.05,
        subsample=0.85, colsample_bytree=0.85, num_leaves=100,
        min_child_samples=20, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1, verbose=-1
    ),
    'Random Forest': RandomForestRegressor(
        n_estimators=300, max_depth=15, min_samples_split=10,
        min_samples_leaf=5, max_features='sqrt',
        random_state=42, n_jobs=-1
    )
}

In [ ]:
# %%
# SECTION 11.5: HYPERPARAMETER TUNING (OPTIONAL)
# =============================================================================
print("\n🔧 HYPERPARAMETER TUNING SETUP")
print("-" * 50)

from sklearn.model_selection import RandomizedSearchCV

# HYPERPARAMETER TUNING GRIDS
# =============================================================================

# XGBoost tuning grid
xgb_param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.03, 0.05, 0.1],
    'subsample': [0.8, 0.85, 0.9],
    'colsample_bytree': [0.8, 0.85, 0.9],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0.5, 1.0, 2.0],
    'min_child_weight': [1, 3, 5]
}

# LightGBM tuning grid
lgb_param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.03, 0.05, 0.1],
    'subsample': [0.8, 0.85, 0.9],
    'colsample_bytree': [0.8, 0.85, 0.9],
    'num_leaves': [50, 100, 150],
    'min_child_samples': [10, 20, 30],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0.5, 1.0, 2.0]
}

# Random Forest tuning grid
rf_param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 5, 10],
    'max_features': ['sqrt', 'log2', 0.8],
    'max_samples': [0.7, 0.8, 0.9]
}

# Hyperparameter tuning function
def tune_hyperparameters(model, param_grid, X_train, y_train, groups, model_name):
    """
    Perform hyperparameter tuning with GroupKFold
    """
    print(f"\n🔧 Tuning {model_name} hyperparameters...")
    
    # Setup GroupKFold
    gkf = GroupKFold(n_splits=4)
    
    # RandomizedSearchCV for efficiency
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=50,              # Number of parameter combinations to try
        cv=gkf,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    # Fit the search
    random_search.fit(X_train, y_train, groups=groups)
    
    print(f"   ✅ Best {model_name} score: {-random_search.best_score_:.4f}")
    print(f"   ✅ Best {model_name} params: {random_search.best_params_}")
    
    return random_search.best_estimator_, random_search.best_params_

print("✅ Hyperparameter tuning setup complete")

# %%
# SECTION 11.6: APPLY HYPERPARAMETER TUNING (OPTIONAL - COMMENT OUT IF SKIPPING)
# =============================================================================
print("\n🚀 APPLYING HYPERPARAMETER TUNING")
print("-" * 50)

# Set this to True if you want to run hyperparameter tuning (takes longer)
RUN_HYPERPARAMETER_TUNING = True  # Change to True to enable

if RUN_HYPERPARAMETER_TUNING:
    print("🔧 Running hyperparameter tuning (this may take 10-30 minutes)...")
    
    # Create base models for tuning
    base_models = {
        'XGBoost': xgb.XGBRegressor(random_state=42, n_jobs=-1),
        'LightGBM': lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
        'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1)
    }
    
    # Parameter grids
    param_grids = {
        'XGBoost': xgb_param_grid,
        'LightGBM': lgb_param_grid,
        'Random Forest': rf_param_grid
    }
    
    # Tune each model
    tuned_models = {}
    tuned_params = {}
    
    for model_name, base_model in base_models.items():
        print(f"\n{'='*60}")
        print(f"TUNING {model_name.upper()}")
        print(f"{'='*60}")
        
        tuned_model, best_params = tune_hyperparameters(
            base_model,
            param_grids[model_name],
            X_train_scaled, y_train, groups_train,
            model_name
        )
        
        tuned_models[model_name] = tuned_model
        tuned_params[model_name] = best_params
    
    # Replace original models with tuned models
    models = tuned_models
    print(f"\n✅ Hyperparameter tuning complete! Using tuned models.")
    
    # Save tuned parameters for reference
    import json
    with open(data_path + '/best_hyperparameters.json', 'w') as f:
        # Convert numpy types to native Python types for JSON serialization
        json_params = {}
        for model_name, params in tuned_params.items():
            json_params[model_name] = {k: int(v) if isinstance(v, np.integer) else 
                                     float(v) if isinstance(v, np.floating) else v 
                                     for k, v in params.items()}
        json.dump(json_params, f, indent=2)
    print(f"   💾 Best parameters saved to: best_hyperparameters.json")

else:
    print("⏭️  Skipping hyperparameter tuning (using default parameters)")
    print("   💡 Set RUN_HYPERPARAMETER_TUNING = True to enable tuning")

In [ ]:
# %%
# SECTION 12: CROSS-VALIDATION AND MODEL EVALUATION
# =============================================================================
print("\n📊 CROSS-VALIDATION EVALUATION")
print("-" * 50)

def evaluate_model_cv(model, X, y, groups, model_name):
    """
    Evaluate model using GroupKFold cross-validation
    """
    print(f"\n   🔄 Evaluating {model_name}...")

    # Cross-validation scores
    cv_scores = cross_val_score(
        model, X, y,
        groups=groups,
        cv=gkf,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    # Convert to RMSE
    cv_rmse = np.sqrt(-cv_scores)

    # Calculate R² using cross-validation
    cv_r2_scores = cross_val_score(
        model, X, y,
        groups=groups,
        cv=gkf,
        scoring='r2',
        n_jobs=-1
    )

    print(f"      📈 CV RMSE: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}")
    print(f"      📈 CV R²: {cv_r2_scores.mean():.4f} ± {cv_r2_scores.std():.4f}")

    return {
        'cv_rmse_mean': cv_rmse.mean(),
        'cv_rmse_std': cv_rmse.std(),
        'cv_r2_mean': cv_r2_scores.mean(),
        'cv_r2_std': cv_r2_scores.std()
    }

# Evaluate all models
cv_results = {}
for model_name, model in models.items():
    cv_results[model_name] = evaluate_model_cv(model, X_train_scaled, y_train, groups_train, model_name)


In [ ]:
# %%
# SECTION 13: FINAL MODEL TRAINING AND VALIDATION
# =============================================================================
print("\n🎯 FINAL MODEL TRAINING")
print("-" * 50)

def train_and_evaluate_final(model, X_train, y_train, X_valid, y_valid, model_name):
    """
    Train final model and evaluate on validation set
    """
    print(f"\n   🚀 Training final {model_name}...")

    # Train model
    model.fit(X_train, y_train)

    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_valid = model.predict(X_valid)

    # Metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    valid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))

    train_r2 = r2_score(y_train, y_pred_train)
    valid_r2 = r2_score(y_valid, y_pred_valid)

    train_mae = mean_absolute_error(y_train, y_pred_train)
    valid_mae = mean_absolute_error(y_valid, y_pred_valid)

    print(f"      📊 Training   - RMSE: {train_rmse:.2f}, R²: {train_r2:.4f}, MAE: {train_mae:.2f}")
    print(f"      📊 Validation - RMSE: {valid_rmse:.2f}, R²: {valid_r2:.4f}, MAE: {valid_mae:.2f}")

    return {
        'model': model,
        'train_rmse': train_rmse, 'valid_rmse': valid_rmse,
        'train_r2': train_r2, 'valid_r2': valid_r2,
        'train_mae': train_mae, 'valid_mae': valid_mae,
        'y_pred_valid': y_pred_valid
    }

# Train all final models
final_results = {}
for model_name, model in models.items():
    final_results[model_name] = train_and_evaluate_final(
        model, X_train_scaled, y_train, X_valid_scaled, y_valid, model_name
    )

In [ ]:
# %%
# SECTION 14: MODEL COMPARISON AND SELECTION
# =============================================================================
print("\n🏆 MODEL COMPARISON SUMMARY")
print("-" * 50)

# Create comparison DataFrame
comparison_data = []
for model_name in models.keys():
    cv_res = cv_results[model_name]
    final_res = final_results[model_name]

    comparison_data.append({
        'Model': model_name,
        'CV_R2_Mean': cv_res['cv_r2_mean'],
        'CV_R2_Std': cv_res['cv_r2_std'],
        'CV_RMSE_Mean': cv_res['cv_rmse_mean'],
        'Valid_R2': final_res['valid_r2'],
        'Valid_RMSE': final_res['valid_rmse'],
        'Valid_MAE': final_res['valid_mae']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Valid_R2', ascending=False)

print("\n📊 PERFORMANCE COMPARISON:")
print(comparison_df.round(4).to_string(index=False))

# Select best model
best_model_name = comparison_df.iloc[0]['Model']
best_model_results = final_results[best_model_name]

print(f"\n🥇 BEST MODEL: {best_model_name}")
print(f"   📈 Validation R²: {best_model_results['valid_r2']:.4f}")
print(f"   📈 Validation RMSE: {best_model_results['valid_rmse']:.2f}")
print(f"   📈 Validation MAE: {best_model_results['valid_mae']:.2f}")


In [ ]:
# %%
# SECTION 15: FINAL TEST EVALUATION
# =============================================================================
print("\n🎯 FINAL TEST EVALUATION")
print("-" * 50)

# Evaluate best model on test set
best_model = best_model_results['model']
y_pred_test = best_model.predict(X_test_scaled)

# Test metrics
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"🏆 FINAL TEST PERFORMANCE ({best_model_name}):")
print(f"   📊 Test RMSE: {test_rmse:.2f}")
print(f"   📊 Test R²: {test_r2:.4f}")
print(f"   📊 Test MAE: {test_mae:.2f}")

# Target statistics comparison
print(f"\n📈 TARGET DISTRIBUTION COMPARISON:")
print(f"   Training   - Mean: ${y_train.mean():,.2f}, Std: ${y_train.std():,.2f}")
print(f"   Validation - Mean: ${y_valid.mean():,.2f}, Std: ${y_valid.std():,.2f}")
print(f"   Test       - Mean: ${y_test.mean():,.2f}, Std: ${y_test.std():,.2f}")
print(f"   Predictions- Mean: ${y_pred_test.mean():,.2f}, Std: ${y_pred_test.std():,.2f}")

print("\n" + "=" * 80)
print("🎉 INCOME PREDICTION MODEL PIPELINE COMPLETE!")
print(f"🎯 FINAL PERFORMANCE: R² = {test_r2:.4f}")
print("=" * 80)


In [ ]:
# %%
# SECTION 16: SAVE RESULTS AND ARTIFACTS
# =============================================================================
print("\n💾 SAVING RESULTS")
print("-" * 50)

# Save model artifacts
import joblib

# Save best model and scaler
model_artifacts = {
    'best_model': best_model,
    'scaler': scaler,
    'feature_columns': feature_columns,
    'model_name': best_model_name,
    'test_performance': {
        'r2': test_r2,
        'rmse': test_rmse,
        'mae': test_mae
    }
}

# Save to file
joblib.dump(model_artifacts, data_path + '/income_prediction_model.pkl')
print(f"   ✅ Model artifacts saved to: income_prediction_model.pkl")

# Save enhanced datasets
train_df_enhanced.to_csv(data_path + '/train_enhanced.csv', index=False)
valid_df_enhanced.to_csv(data_path + '/valid_enhanced.csv', index=False)
test_df_enhanced.to_csv(data_path + '/test_enhanced.csv', index=False)
print(f"   ✅ Enhanced datasets saved")

# Save comparison results
comparison_df.to_csv(data_path + '/model_comparison.csv', index=False)
print(f"   ✅ Model comparison saved")

print("\n🎯 NEXT STEPS FOR IMPROVEMENT:")
print("   1. Hyperparameter tuning with RandomizedSearchCV")
print("   2. Ensemble methods (combine top models)")
print("   3. Additional feature engineering")
print("   4. Advanced outlier detection methods")
print(f"   5. Target: Improve R² from {test_r2:.4f} to 0.35+")

print("\n✅ PIPELINE READY FOR OPTIMIZATION!")
print("=" * 80)

In [ ]:
# The best model is already available in your session
print(f"Best model: {best_model_name}")
print(f"Model object: {best_model}")

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
# Calculate permutation importance
perm_importance = permutation_importance(best_model, X_test_scaled, y_test, n_repeats=15, random_state=42)

In [ ]:
# Sort features by importance
feature_importance = perm_importance.importances_mean
sorted_idx = feature_importance.argsort()

In [ ]:
# Get feature names from your dataset (replace these with your actual feature names)
feature_names = X_test_scaled.columns  # or your list of feature names

In [ ]:
# Create the plot
plt.figure(figsize=(8, 4))

# Get the top 15 (or fewer) important features
num_features_to_plot = min(20, len(feature_importance))
top_indices = sorted_idx[-num_features_to_plot:]

# Plot only the top features
plt.barh(range(num_features_to_plot), feature_importance[top_indices],color='red')
plt.yticks(range(num_features_to_plot), [feature_names[i] for i in top_indices])
plt.xlabel('Mean Decrease in Accuracy')
plt.title('Permutation Importance (Top 20 Features)')
plt.tight_layout()
plt.show()

# Print the names of the top features
print("Top 20 Feature Names:")
for i in reversed(top_indices):
    print(feature_names[i])

In [ ]:
y_test_pred_lg_class = best_model.predict(X_test_scaled)

In [ ]:
test_lg_test =X_test_scaled.copy()
test_lg_test['target'] = y_test
test_lg_test ["set_type"]='test'

In [ ]:
test_lg_test["pred_income"] = y_test_pred_lg_class

In [ ]:
# Calculate performance metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

mae = mean_absolute_error(test_lg_test.target, test_lg_test.pred_income)
mse = mean_squared_error(test_lg_test.target, test_lg_test.pred_income)
rmse = np.sqrt(mse)
r2 = r2_score(test_lg_test.target, test_lg_test.pred_income)
evs = explained_variance_score(test_lg_test.target, test_lg_test.pred_income)

In [ ]:
# Calculate MAPE
mape = np.mean(np.abs((test_lg_test.target - test_lg_test.pred_income) / test_lg_test.target)) * 100

In [ ]:
# Print the results
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²) Score: {r2:.4f}")
print(f"Explained Variance Score: {evs:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")

In [ ]:
#best_model_name = best_model
X_test_final = X_test_scaled
y_train_cleaned = y_train
y_test_cleaned = y_test
best_model_name = best_model

In [ ]:
# Visualizing Training Target vs Test Predictions
# ============================================================================
print("=" * 60)
print("VISUALIZING TRAINING TARGET vs TEST PREDICTIONS")
print("=" * 60)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# First, get predictions on test set using the best model
best_model_obj = best_model  # Use best_model_name instead of best_model
test_predictions = best_model_obj.predict(X_test_final)

print(f"Using best model: {best_model_name}")
print(f"Test predictions shape: {test_predictions.shape}")
print(f"Training target shape: {y_train_cleaned.shape}")

# Set up the plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle(f'Training Target vs Test Predictions - {best_model_name}', fontsize=16)

# 1. Distribution Comparison (Histograms)
axes[0,0].hist(y_train_cleaned, bins=50, alpha=0.7, label='Training Target', color='blue', density=True)
axes[0,0].hist(test_predictions, bins=50, alpha=0.7, label='Test Predictions', color='red', density=True)
axes[0,0].set_xlabel('Income (ingresos_reportados)')
axes[0,0].set_ylabel('Density')
axes[0,0].set_title('Distribution Comparison')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Box Plot Comparison
box_data = [y_train_cleaned, test_predictions]
box_labels = ['Training Target', 'Test Predictions']
axes[0,1].boxplot(box_data, labels=box_labels)
axes[0,1].set_ylabel('Income (ingresos_reportados)')
axes[0,1].set_title('Box Plot Comparison')
axes[0,1].grid(True, alpha=0.3)

# 3. Q-Q Plot (Quantile-Quantile)
train_quantiles = np.percentile(y_train_cleaned, np.linspace(0, 100, 100))
pred_quantiles = np.percentile(test_predictions, np.linspace(0, 100, 100))

axes[0,2].scatter(train_quantiles, pred_quantiles, alpha=0.6)
axes[0,2].plot([min(train_quantiles), max(train_quantiles)], 
               [min(train_quantiles), max(train_quantiles)], 'r--', lw=2)
axes[0,2].set_xlabel('Training Target Quantiles')
axes[0,2].set_ylabel('Test Predictions Quantiles')
axes[0,2].set_title('Q-Q Plot')
axes[0,2].grid(True, alpha=0.3)

# 4. Actual vs Predicted Scatter Plot (Test Set)
axes[1,0].scatter(y_test_cleaned, test_predictions, alpha=0.6, s=20)
axes[1,0].plot([y_test_cleaned.min(), y_test_cleaned.max()], 
               [y_test_cleaned.min(), y_test_cleaned.max()], 'r--', lw=2)
axes[1,0].set_xlabel('Actual Test Values')
axes[1,0].set_ylabel('Predicted Test Values')
axes[1,0].set_title('Actual vs Predicted (Test Set)')
axes[1,0].grid(True, alpha=0.3)

# Calculate R² for the plot
test_r2 = r2_score(y_test_cleaned, test_predictions)
axes[1,0].text(0.05, 0.95, f'R² = {test_r2:.3f}', transform=axes[1,0].transAxes, 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 5. Residuals Plot
residuals = y_test_cleaned - test_predictions
axes[1,1].scatter(test_predictions, residuals, alpha=0.6, s=20)
axes[1,1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1,1].set_xlabel('Predicted Values')
axes[1,1].set_ylabel('Residuals (Actual - Predicted)')
axes[1,1].set_title('Residuals Plot')
axes[1,1].grid(True, alpha=0.3)

# 6. Cumulative Distribution Functions
train_sorted = np.sort(y_train_cleaned)
pred_sorted = np.sort(test_predictions)
train_cdf = np.arange(1, len(train_sorted) + 1) / len(train_sorted)
pred_cdf = np.arange(1, len(pred_sorted) + 1) / len(pred_sorted)

axes[1,2].plot(train_sorted, train_cdf, label='Training Target', linewidth=2)
axes[1,2].plot(pred_sorted, pred_cdf, label='Test Predictions', linewidth=2)
axes[1,2].set_xlabel('Income (ingresos_reportados)')
axes[1,2].set_ylabel('Cumulative Probability')
axes[1,2].set_title('Cumulative Distribution Functions')
axes[1,2].legend()
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical Comparison
print(f"\n--- STATISTICAL COMPARISON ---")
print(f"Training Target Statistics:")
print(f"  - Mean: ${y_train_cleaned.mean():,.2f}")
print(f"  - Median: ${y_train_cleaned.median():,.2f}")
print(f"  - Std: ${y_train_cleaned.std():,.2f}")
print(f"  - Min: ${y_train_cleaned.min():,.2f}")
print(f"  - Max: ${y_train_cleaned.max():,.2f}")

print(f"\nTest Predictions Statistics:")
print(f"  - Mean: ${test_predictions.mean():,.2f}")
print(f"  - Median: ${np.median(test_predictions):,.2f}")
print(f"  - Std: ${test_predictions.std():,.2f}")
print(f"  - Min: ${test_predictions.min():,.2f}")
print(f"  - Max: ${test_predictions.max():,.2f}")

print(f"\nTest Set Performance:")
test_rmse = np.sqrt(mean_squared_error(y_test_cleaned, test_predictions))
test_mae = np.mean(np.abs(y_test_cleaned - test_predictions))
print(f"  - RMSE: {test_rmse:.4f}")
print(f"  - R²: {test_r2:.4f}")
print(f"  - MAE: {test_mae:.4f}")

# Distribution similarity tests
from scipy.stats import ks_2samp
ks_statistic, ks_pvalue = ks_2samp(y_train_cleaned, test_predictions)
print(f"\nKolmogorov-Smirnov Test:")
print(f"  - KS Statistic: {ks_statistic:.4f}")
print(f"  - P-value: {ks_pvalue:.4f}")
print(f"  - Interpretation: {'Distributions are similar' if ks_pvalue > 0.05 else 'Distributions are different'}")